In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from tensorflow.keras.callbacks import EarlyStopping # type: ignore
from sklearn.impute import SimpleImputer

In [2]:
# Data Loading
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

# Label Separation
X = train.drop(columns = ['ID', 'SUBCLASS'], axis=1)
y = train['SUBCLASS']

In [3]:
# 결측치 처리
imputer = SimpleImputer(strategy='most_frequent')
test = test.drop(columns = 'ID')
test = pd.DataFrame(imputer.fit_transform(test), columns=test.columns)

In [4]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

# train.csv 
oe = OrdinalEncoder()
X_df = oe.fit_transform(X)
data = pd.DataFrame(X_df).copy()
data.columns = X.columns

# test.csv
temporary_columns = test.columns
test = pd.DataFrame(oe.fit_transform(test))
test.columns = temporary_columns


# y Data
le = LabelEncoder()
y = le.fit_transform(y)

In [7]:
# Macro F1 Score를 계산하는 사용자 정의 메트릭
class MacroF1Score(tf.keras.metrics.Metric):
    def __init__(self, num_classes, name='macro_f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.true_positives = self.add_weight(name='tp', initializer='zeros', shape=(num_classes,))
        self.false_positives = self.add_weight(name='fp', initializer='zeros', shape=(num_classes,))
        self.false_negatives = self.add_weight(name='fn', initializer='zeros', shape=(num_classes,))

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.reshape(y_true, [-1])
        y_pred = tf.reshape(tf.argmax(y_pred, axis=1), [-1])
        y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=self.num_classes)
        y_pred = tf.one_hot(tf.cast(y_pred, tf.int32), depth=self.num_classes)
        
        self.true_positives.assign_add(tf.reduce_sum(y_true * y_pred, axis=0))
        self.false_positives.assign_add(tf.reduce_sum((1 - y_true) * y_pred, axis=0))
        self.false_negatives.assign_add(tf.reduce_sum(y_true * (1 - y_pred), axis=0))

    def result(self):
        precision = self.true_positives / (self.true_positives + self.false_positives + tf.keras.backend.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())
        f1 = 2 * precision * recall / (precision + recall + tf.keras.backend.epsilon())
        return tf.reduce_mean(f1)

    def reset_state(self):
        self.true_positives.assign(tf.zeros_like(self.true_positives))
        self.false_positives.assign(tf.zeros_like(self.false_positives))
        self.false_negatives.assign(tf.zeros_like(self.false_negatives))

In [8]:
# 모델 정의
from tensorflow import keras # type: ignore
from tensorflow.keras import layers # type: ignore
model = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=(data.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(26, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=[MacroF1Score(num_classes=26)])


C:\Users\seren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# 모델 학습
history = model.fit(
    data, y,
    epochs=100,
    batch_size=32
)

Epoch 1/100
194/194 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 3.6103 - macro_f1_score: 0.0685
Epoch 2/100
194/194 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 2.7395 - macro_f1_score: 0.1456
Epoch 3/100
194/194 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 2.4027 - macro_f1_score: 0.1892
Epoch 4/100
194/194 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 2.2258 - macro_f1_score: 0.2296
Epoch 5/100
194/194 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 2.0715 - macro_f1_score: 0.2715
Epoch 6/100
194/194 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.8800 - macro_f1_score: 0.3132
Epoch 7/100
194/194 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.7535 - macro_f1_score: 0.3662
Epoch 8/100
194/194 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.6426 - macro_f1_score: 0.4015
Epoch 9/100
194/194 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.5371 - macro_f1_score: 0.4402
Epoch 10/100
194/194 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.4852 - macro_f1_score: 0.4566
Epoch 11/100
194/194 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.3733 

In [10]:
# 테스트 데이터 예측
predictions = model.predict(test)
predicted_classes = np.argmax(predictions, axis=1)

80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [12]:
# 예측 결과를 원래 레이블로 변환
predicted_labels = le.inverse_transform(predicted_classes)


In [14]:
result = pd.DataFrame(predicted_labels)

final = sample.copy()
final['SUBCLASS'] = result

In [22]:
final.to_csv("./final.csv", index=False)